# 最適化（多変数）

- すべての課題を実行せよ
- すべての課題が完了したら Jupyter の File メニューから Download as -> Markdown (.md) として結果をダウンロードし， Bb9 の「課題」からファイル添付で提出せよ

In [ ]:
# 初期化なので最初に実行してください
include("../lib/Trace.jl")
using LinearAlgebra
using .Trace
using Test
using Printf

## 課題1

### 勾配降下法（1変数）
- アルミホ条件がなりたつように値が小さくなる方向へ探索

## 課題1-1
- バックトラックによる直線探索法のプログラムを完成させよ


### linesearch の説明
- 引数
    - f: 非線形関数．f(x) のように呼び出される
    - d: 降下方向
    - x0: $f(x0+eta*d)$
    - f0: $f0 = f(x0)$
    - fdash0: $fdash0 = f'(x0)$
    - xi1: 降下条件の緩和パラメータ
    - beta: eta の減少率
    - eta0: eta の最大値
- 戻り値
    - 降下条件を満たす　$x1, f(x1)$ の値
- 関数内の変数
    - eta: $f(x0 + eta * d)$ 
    - x0, x1: 現在の点，次の点

In [ ]:
function linesearch(f, d, x0, f0, fdash0; xi1=1/10000, beta=0.8, eta0=1.0)
    eta = eta0
    @trace x1, f1 = x0 + eta * d, f(x0 + eta * d)
    while _______________________ # 降下条件がなりたつまで eta を減少する
        eta *= beta
        @trace x1, f1 = x0 + eta * d, f(x0 + eta * d)
    end
    return (x1, f1)
end

#### テストプログラム
- 作成できたら以下のテストプログラムを実行してすべて**Test Passed**になることを確認
- **Test Failed** が出たら結果が計算が間違っているので修正

In [ ]:
f(x) = x^2-3x+2
fdash(x) = 2x-3
x0 = 0.1
x1, f1 = linesearch(f, -fdash(x0), x0, f(x0), fdash(x0))
println(@test x1 ≈ 2.34)
println(@test f1 ≈ 0.45559999999999956)
x0 = 1.5
x1, f1 = linesearch(f, -fdash(x0), x0, f(x0), fdash(x0))
println(@test x1 ≈ 1.5)
println(@test f1 ≈ -0.25)

## 課題1-2
- xi1を変えた場合の探索の違いを確認し，なぜ違うのか考察せよ．また $f(x)=x^2-3x+2$ のときにx0=3, xi1=1とするとアルミホ条件を満たすx1は理論的にどうなるか答えよ

In [ ]:
f(x) = x^2-3x+2
fdash(x) = 2x-3
x0 = 0.1

xi1 = 0.1
@traceon x1, f1 = linesearch(f, -fdash(x0), x0, f(x0), fdash(x0), xi1 = xi1)
@printf("\nxi1=%f としたときの探索\n", xi1)
for (i,x) in enumerate(@gettrace)
    @printf("step=%d: f(%f) = %f\n", i, x[1], x[2])
end

xi1 = 0.5
@traceon x1, f1 = linesearch(f, -fdash(x0), x0, f(x0), fdash(x0), xi1 = xi1)
@printf("\nxi1=%f としたときの探索\n", xi1)
for (i,x) in enumerate(@gettrace)
    @printf("step=%d: f(%f) = %f\n", i, x[1], x[2])
end

#### 解答

- ここに理由を記述
- xi1=1としたときのx1の値

## 課題1-3
- 勾配降下法のプログラムを定義し，テストプログラムを実行せよ

### gd の説明
- 引数
    - f: 非線形関数．f(x) のように呼び出される
    - fdash: 導関数．fdash(x)のように呼び出される
    - x0: 初期値
    - xi1: 降下条件の緩和パラメータ
    - beta: eta の減少率
    - eta0: eta の最大値
    - tol: 許容誤差
    - maxiter: 最大繰り返し回数
- 戻り値
    - 最小解，最小値,その時の微分値
    - 収束したかどうか
    - 繰り返し回数
- 関数内の変数
    - iter: 繰返し回数
    - d: 降下方向
    - eta: $f(x0 + eta * d)$ 
    - x0, f0, fdash0: 現在の点x0におけるf0=f(x0)およびfdash0=fdash(x0)
    - x1, f1, fdash1: 次の点x1におけるf1=f(x1)およびfdash1=fdash(x1)

In [ ]:
function gd(f, fdash, x0; xi1=1/10000, beta=0.8, eta0=1.0, tol=1.0e-8, maxiter=10000)
    iter = 0
    f0, fdash0 = f(x0), fdash(x0)
    while true
        d = -fdash0
        x1, f1 = linesearch(f, d, x0, f0, fdash0, xi1=xi1, beta=beta, eta0=eta0)
        fdash1 = fdash(x1)

        abs(fdash1) < tol && return (x1, f1, fdash1, true, iter) # 終了条件その１
        iter >= maxiter && return (x1, f1, fdash1, false, iter) # 終了条件その２

        x0, f0, fdash0 = x1, f1, fdash1
        iter += 1
    end
end

#### テストプログラム
- 作成できたら以下のテストプログラムを実行してすべて**Test Passed**になることを確認
- **Test Failed** が出たら結果が計算が間違っているので修正

In [ ]:
f(x) = x^2-3x+2
fdash(x) = 2x-3
x0 = 0.1
xmin,fmin,fd,conv,iter = gd(f, fdash, x0, beta=0.6)
println(@test xmin ≈ 3/2)

## 課題1-4
- exercise06の課題2-2と同じ関数を用いて $f(x)$ の最小解を gd を使って求めよ
- exercise06の課題1-2,課題2-2の結果と繰返し回数の比較を行え

In [ ]:
# f(x) を定義
f(x) = ?
# fdash(x) を定義
fdash(x) = ?
# 初期値 x0 をセット
x0 = ?
# gd を実行
xmin,fmin,fd,conv,iter = gd(f, fdash, x0)

|手法|繰り返し回数|
|:---:|:---:|
| goldensection | 0 |
| opnewton | 0 |
| gd | 0 |

---

## 課題2
- Nelder-Meadシンプレックス法
- 多変数関数の最小化で微分値を使わずに最小解，最小値を求める手法
- 多面体をコロコロ転がして最小値へ持って行く

### 課題2-1
- Nelder-Meadシンプレックス法のプログラムを作成せよ
- テストプログラムで動作を確認せよ

### nmsimplex の説明
- 引数
    - f: 非線形関数．f(x) のように呼び出される
    - xv: 初期点のリスト
    - alpha: 反射パラメータ
    - gamma: 拡張パラメータ
    - rho: 収縮パラメータ
    - sigma: 縮小パラメータ
    - tol: 許容誤差
    - maxiter: 最大繰り返し回数
- 戻り値
    - 最小解，最小値
    - 収束したかどうか
    - 繰り返し回数
- 関数内の変数
    - iter: 繰返し回数
    - fv: 各 xv における点の評価値（xv, fvは評価値の小さい順にソートされる）
    - xo: xv[1] から xv[n] の点の重心
    - xr, fr: 反射により得られた新たな点とその評価値
    - xe, fe: 拡張により得られた新たな点とその評価値
    - xc, fc: 収縮により得られた新たな点とその評価値

In [ ]:
function nmsimplex(f, xv; alpha = 1.0, gamma = 2.0, rho = 1/2, sigma = 1/2, tol = 1.0e-12, maxiter = 100)
    n = length(xv[1])
    iter = 1
    fv = f.(xv) # ｆ（x_1), f(x_2), ...,　f(x_n) を計算
    while true # 無限ループ
        iter += 1
        # fv の小さい順に xv, fv をソート
        p = sortperm(fv)
        xv, fv = xv[p], fv[p]
        @trace xo = sum(xv[1:n]) / n # 重心を計算

        maximum(abs.(xv[1] - xv[n+1])) < tol &&  return (xv[1], fv[1], true, iter) # 終了条件
        iter >= maxiter && return (xv[1], fv[1], false, iter) 　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　# 終了条件

        ## 反射 reflection
        xr = ___________________　# ? x[n+1] の点を xo に対して反転した点
        fr = f(xr)
        if fv[1] <= fr < fv[n] # 反射したものが f[1] を超えない程度に良かったら最も悪い値を交換
            xv[n+1] = xr
            fv[n+1] = fr
            continue
        end
        
        if fr < fv[1] # 反射したものが1番良かったらさらに良い解を探す
            ## 拡張　expansion
            xe = xo + gamma * (xr - xo)
            fe = f(xe)
            if fe < fr
                xv[n+1], fv[n+1] = xe, fe
            else
                xv[n+1], fv[n+1] = xr, fr
            end
            continue
        end

        ## 収縮 contraction
        xc = xo + rho * (xv[n+1] - xo)
        fc = f(xc)
        if fc < fv[n+1]
            xv[n+1], fv[n+1] = xc, fc
        else
            ## 縮小 reduction/shrink
            for i = 2:n+1
                xv[i] = xv[1] + sigma * (xv[i] - xv[1])
                fv[i] = f(xv[i])
            end
        end
    end
end

### nminitial の説明（これも実行して定義してください）
- 概要
    - 与えられた点から n+1 個の初期点を生成する
    - 各座標毎に lambda を可算する
- 引数
    - x: 初期点（一つ）
    - lambda: 各座標に可算する数値
- 戻り値
    - 最初に与えた x を含む n+1 個の点のリスト
- 関数内の変数（省略）

In [ ]:
function nminitial(x; lambda = 1.0)
    n = length(x)
    list = map(1:n) do i
        s = copy(x)
        s[i] += lambda
        s
    end
    push!(list, x)
    return list
end

#### テストプログラム
- 作成できたら以下のテストプログラムを実行してすべて**Test Passed**になることを確認
- **Test Failed** が出たら結果が計算が間違っているので修正

In [ ]:
f(x) = (x[1]-1.0)^2 + 5*(x[2]-1.0)^2
xmin, fmin, conv, iter = nmsimplex(f, nminitial([10.0, 10.0]))
println(@test xmin ≈ [1.0, 1.0])

### 課題2-2
- ランダムに問題を作成し，最適解が求まるようにNelder-Meadシンプレックス法のパラメータを調整せよ

#### 下記を実行して最適化問題をランダムに作成

In [ ]:
n = 5
A = randn(n, n)
b = randn(n)
A = A' * A
f(x) = x' * (A * x) / 2.0 - b' * x
fdash(x) = A * x - b
xmin0 = A \ b # 最適解

In [ ]:
# テストにパスするように maxiter, alpha, gamma, rho, sigma を調整する
@traceon xmin, fmin, conv, iter = nmsimplex(f, nminitial(fill(0.0, n)), maxiter = 100, alpha = 1.0, gamma = 2.0, rho = 1/2, sigma = 1/2)
path1 = map(x->(x[1], x[2]), @gettrace)
display(path1)
println(@test xmin0 ≈ xmin)

## 課題3
- 勾配降下法

### 課題3-1
- 学習率固定のラフな最急降下法のプログラムを作成せよ
- テストプログラムで動作を確認せよ

### gd1 の説明
- 引数
    - f: 非線形関数．f(x) のように呼び出される
    - fdash: 導関数．fdash(x)のように呼び出される
    - x0: 初期値
    - eta: 学習率（固定値）
    - maxiter: 最大繰り返し回数
- 戻り値
    - 最小解，最小値,その時の微分値
- 関数内の変数
    - iter: 繰返し回数
    - d: 降下方向
    - x0, f0, fdash0: 現在の点x0におけるf0=f(x0)およびfdash0=fdash(x0)
    - x1, f1, fdash1: 次の点x1におけるf1=f(x1)およびfdash1=fdash(x1)

In [ ]:
function gd1(f, fdash, x0; eta = 0.001, maxiter=100)
    iter = 0
    @trace x0, f0, fdash0 = x0, f(x0), fdash(x0)
    while true
        d = -fdash0
        _______________ # 更新処理
        f1, fdash1 = f(x1), fdash(x1)
        if iter >= maxiter # 回数の判定
            return x1, f1, fdash1
        end
        @trace x0, f0, fdash0 = x1, f1, fdash1 # 更新した座標 x1 を現在の座標 x0 にする
        iter += 1
    end
end

#### テストプログラム
- 作成できたら以下のテストプログラムを実行してすべて**Test Passed**になることを確認
- **Test Failed** が出たら結果が計算が間違っているので修正

In [ ]:
f(x) = (x[1]-1.0)^2 + 5*(x[2]-1.0)^2
fdash(x) = [2.0*(x[1] - 1.0), 10.0 * (x[2] - 1.0)]
xmin, fmin, fd = gd1(f, fdash, [0.0, 0.0], eta = 0.1)
println(@test xmin ≈ [1.0, 1.0])

### 課題3-2
- ランダムに問題を作成し，最適解が求まるようにgd1のパラメータを調整せよ

#### 下記を実行して最適化問題をランダムに作成

In [ ]:
n = 5
A = randn(n, n)
b = randn(n)
A = A' * A
f(x) = x' * (A * x) / 2.0 - b' * x
fdash(x) = A * x - b
xmin0 = A \ b # 最適解

In [ ]:
# テストにパスするように学習率 eta と maxiter を調整する
@traceon xmin, fmin, fd = gd1(f, fdash, fill(0.0, n), eta = 0.001, maxiter = 100)
path2 = map(x->(x[1], x[2]), @gettrace)
display(path2)
println(@test xmin0 ≈ xmin)

### 課題3-3
- 降下条件を考慮した最急降下法のプログラムを作成せよ
- テストプログラムで動作を確認せよ

### gd2 の説明
- 引数
    - f: 非線形関数．f(x) のように呼び出される
    - fdash: 導関数．fdash(x)のように呼び出される
    - x0: 初期値
    - xi1: 降下条件の緩和パラメータ
    - beta: eta の減少率
    - eta0: eta の最大値
    - tol: 許容誤差
    - maxiter: 最大繰り返し回数
- 戻り値
    - 最小解，最小値,その時の微分値
    - 収束したかどうか
    - 繰り返し回数
- 関数内の変数
    - iter: 繰返し回数
    - d: 降下方向
    - eta: $f(x0 + eta * d)$ 
    - x0, f0, fdash0: 現在の点x0におけるf0=f(x0)およびfdash0=fdash(x0)
    - x1, f1, fdash1: 次の点x1におけるf1=f(x1)およびfdash1=fdash(x1)

In [ ]:
# ベクトル版 linesearch
function linesearch(f, d, x0, f0, fdash0; xi1=1/10000, beta=0.8, eta0=1.0)
    eta = eta0
    x1, f1 = x0 + eta * d, f(x0 + eta * d)
    while f1 > f0 + xi1 * eta * dot(fdash0, d) # 降下条件がなりたつまで eta を減少する f1 > f0 + xi1 * eta * fdash0 * d
        eta *= beta
        x1, f1 = x0 + eta * d, f(x0 + eta * d)
    end
    return (x1, f1)
end

function gd2(f, fdash, x0; xi1=1/10000, beta=0.8, eta0=1.0, tol=1.0e-12, maxiter=100)
    iter = 0
    @trace x0, f0, fdash0 = x0, f(x0), fdash(x0)
    while true
        d = -fdash0
        x1, f1 = linesearch(f, d, x0, f0, fdash0, xi1=xi1, beta=beta, eta0=eta0)
        fdash1 = fdash(x1)
        maximum(abs.(x1 - x0)) < tol && return (x1, f1, fdash1, true, iter)
        iter >= maxiter && return (x1, f1, fdash1, false, iter)
        @trace x0, f0, fdash0 = x1, f1, fdash1
        iter += 1
    end
end

#### テストプログラム
- 作成できたら以下のテストプログラムを実行してすべて**Test Passed**になることを確認
- **Test Failed** が出たら結果が計算が間違っているので修正

In [ ]:
f(x) = (x[1]-1.0)^2 + 5*(x[2]-1.0)^2
fdash(x) = [2.0*(x[1] - 1.0), 10.0 * (x[2] - 1.0)]
xmin, fmin, fd = gd2(f, fdash, [0.0, 0.0])
println(@test xmin ≈ [1.0, 1.0])

### 課題3-4
- ランダムに問題を作成し，最適解が求まるようにgd2のパラメータを調整せよ

#### 下記を実行して最適化問題をランダムに作成

In [ ]:
n = 5
A = randn(n, n)
b = randn(n)
A = A' * A
f(x) = x' * (A * x) / 2.0 - b' * x
fdash(x) = A * x - b
xmin0 = A \ b # 最適解

In [ ]:
# テストにパスするようxi1, beta, eta0, maxiter を調整する
@traceon xmin, fmin, fd = gd2(f, fdash, fill(0.0, n), xi1=1/10000, beta=0.8, eta0=1.0, maxiter=100)
path3 = map(x->(x[1], x[2]), @gettrace)
display(path3)
println(@test xmin0 ≈ xmin)

## 課題4
- 準ニュートン法

### 課題4-1
- 準ニュートン法のプログラムを作成せよ
- テストプログラムで動作を確認せよ

In [ ]:
function bfgs(f, fdash, x0; xi1=1/10000, beta=0.8, eta0=1.0,  tol=1.0e-12, maxiter=100)
    iter = 0
    n = length(x0)
    B = Matrix{Float64}(I, n, n) # 単位行列
    @trace x0, f0, fdash0 = x0, f(x0), fdash(x0)
    while true
        d = _____________ # コードを記述
        x1, f1 = linesearch(f, d, x0, f0, fdash0, xi1=xi1, beta=beta, eta0=eta0)
        fdash1 = fdash(x1)

        maximum(abs.(x1 - x0)) < tol && return (x1, f1, fdash1, true, iter)
        iter >= maxiter && return (x1, f1, fdash1, false, iter)

        s = _____________ # コードを記述
        y = _____________ # コードを記述
        v = B * s
        B = B - (v * v') / dot(v, s) + (y * y') / dot(s, y)
        @trace x0, f0, fdash0 = x1, f1, fdash1
        iter += 1
    end
end

#### テストプログラム
- 作成できたら以下のテストプログラムを実行してすべて**Test Passed**になることを確認
- **Test Failed** が出たら結果が計算が間違っているので修正

In [ ]:
f(x) = (x[1]-1.0)^2 + 5*(x[2]-1.0)^2
fdash(x) = [2.0*(x[1] - 1.0), 10.0 * (x[2] - 1.0)]
xmin, fmin, fd = bfgs(f, fdash, [0.0, 0.0])
println(@test xmin ≈ [1.0, 1.0])

### 課題4-2
- ランダムに問題を作成し，最適解が求まるようにbfgsのパラメータを調整せよ

#### 下記を実行して最適化問題をランダムに作成

In [ ]:
n = 5
A = randn(n, n)
b = randn(n)
A = A' * A
f(x) = x' * (A * x) / 2.0 - b' * x
fdash(x) = A * x - b
xmin0 = A \ b # 最適解

In [ ]:
# テストにパスするようxi1, beta, eta0 を調整する
@traceon xmin, fmin, fd = bfgs(f, fdash, fill(0.0, n), xi1=1/10000, beta=0.8, eta0=1.0)
path4 = map(x->(x[1], x[2]), @gettrace)
display(path4)
println(@test xmin0 ≈ xmin)

In [ ]:
f(x) = 100.0*(x[2] - x[1]^2)^2 + (1 - x[1])^2
fdash(x) = [-200.0*(x[2] -x[1]^2)*2.0*x[1] - 2.0(1-x[1]), 200.0*(x[2] - x[1]^2)]
xmin, fmin, conv, iter = nmsimplex(f, nminitial([10.0, 10.0]))
println(@test xmin ≈ [1.0, 1.0])

## 課題5
- 最適化実践
- 以下の Rosenbrock 関数を最小にする解を何らかの手法を使って求めよ

In [ ]:
f(x) = 100.0*(x[2] - x[1]^2)^2 + (1 - x[1])^2

In [ ]:
# ここにコードを記述
